In [ ]:
import os
import pandas as pd

In [ ]:
os.chdir('C://Users//USER//gs크롤링')

In [ ]:
keyword = input()
key = keyword.split(' ')
key

In [ ]:
keyword = input()
key = keyword.split(' ')
key

In [ ]:
# 키워드 별 크롤링 엑셀 파일들 하나로 합치기
# 크롤링 된 파일에 키워드 column 없으면 추가

for key in key:
    i = 1
    data = pd.DataFrame(columns = ['keyword', 'text', 'date'])
    try:
        os.chdir('C://Users//USER//gs크롤링')
        while True:
            data1 = pd.read_csv(key + str(i) + '.csv', encoding='cp949')
            del data1['Unnamed: 0']
            if len(data1.columns) == 2:
                data1.insert(0, 'keyword', key)
            data1.columns = ['keyword', 'text', 'date']
            data = pd.concat([data, data1])
            i+=1
    except:
        print('')
    data.reset_index(drop = True, inplace = True)
    os.chdir('C://Users//USER//gs크롤링 최종')
    data.to_csv(key + '.csv', encoding='cp949')

In [ ]:
os.chdir('C://Users//USER//gs크롤링 최종')

In [ ]:
keyword = input()
key = keyword.split(' ')
key

In [ ]:
data = pd.DataFrame(columns = ['keyword', 'text', 'date'])

In [ ]:
# 키워드 별 크롤링 된 파일 하나로 합치기

for key in key:
    data1 = pd.read_csv(key + '.csv', encoding='cp949')
    data = pd.concat([data, data1])
data.reset_index(drop = True, inplace = True)
del data['Unnamed: 0']

In [ ]:
sheet = data

In [ ]:
sheet

In [ ]:
sheet.to_csv('직장인_크롤링_데이터.csv')

In [ ]:
sheet = pd.read_csv('직장인_크롤링_데이터.csv')

In [ ]:
del sheet['Unnamed: 0']

In [ ]:
sheet

In [ ]:
sheet.keyword.unique()

### Null값 확인

In [ ]:
#혹시 Null 값을 가진 데이터가 있는지 확인 
print(sheet.isnull().sum())

In [ ]:
# 결측치 존재하는 행 제거
sheet.dropna(axis=0, inplace = True)

In [ ]:
# index 순서 리셋
sheet = sheet.reset_index(drop=True)

In [ ]:
#혹시 Null 값을 가진 데이터가 있는지 확인 
print(sheet.isnull().sum())

In [ ]:
sheet.keyword.unique()

### 중복데이터 확인/제거

In [ ]:
#중복 데이터 개수 확인
sheet['text'].duplicated().sum()

In [ ]:
#먄약 중복 데이터가 있다면 제거
sheet = sheet.drop_duplicates(['text'])
sheet.reset_index(inplace = True)
del sheet['index']
# sheet = sheet.iloc[:,[2]] #review column만 가져오기
sheet

In [ ]:
sheet.keyword.unique()

### 2022, 2023년 현재 업로드한 게시물만 사용
- 22년 1월~ 23년 3월(현재) : 15개월

In [ ]:
import re
def check_date(date):
    date_reg = re.compile('2022|2023|어제|일|시간')
    date_bool = date_reg.search(date) != None
    return date_bool
date_check = sheet.date.apply(check_date)
date_check

In [ ]:
sheet = sheet[date_check].reset_index(drop = True)
print(sheet.shape)
sheet.head()

### stopwors 삭제

In [ ]:
#광고글 특징을 Stopwords로 지정하여 해당 글은 삭제할 수 있도록 함
#Stopwords에 해당 문구가 들어가면 삭제 대상이 되도록 문구를 추가해줌


Stopwords = ['모집중', '사례비', '국평오', '실거래가 등록 기준', '오늘의 운세', '블라 형들의 맛집 리스트',
            '경제살롱', '신문 요약', '뉴스 운세', '뉴스운세', '운세뉴스', '설문, 인터뷰', '설문,인터뷰',
            '장중 특징주, 장중특징주'] # 광고 관련 키워드 추가

In [ ]:
sheet['text'].str.contains('|'.join(Stopwords))

In [ ]:
#Stopwords가 들어간 글은 삭제

clean_sheet = sheet[~sheet['text'].str.contains('|'.join(Stopwords))]
clean_sheet

In [ ]:
#특정 '단어' 삭제
def clean_str(text):
    remove_w = ['국평오', '']
    for i in remove_w:
            if i in text:
                print(i)
                text = text.replace(i, ' ')
    return text

In [ ]:
clean_sheet['text'] = clean_sheet['text'].apply(lambda x : clean_str(x))

### 정규식 -1

In [ ]:
import re

def extract_word(text) :
    hangul = re.compile('[^가-힣a-zA-Z]')
    result = hangul.sub(' ', text)
    return result

In [ ]:
clean_sheet['text'] = clean_sheet['text'].apply(lambda x : extract_word(x))

### 정규식-2

In [ ]:
import re

a = re.compile("\S*@\S+")
b = re.compile("#\S+")
c = re.compile("https?://\S+")
d = re.compile("www\.\S+")

def cleanse(text):
    result = a.sub("", text)
    result = b.sub("", result)
    result = c.sub("", result)
    result = d.sub("", result)
    result = result.replace("\u200b", "")
    result = result.replace("\n", "")
    return result

In [ ]:
clean_sheet['text'] = clean_sheet['text'].apply(lambda x : cleanse(x))

In [ ]:
clean_sheet.reset_index(drop=True, inplace = True)

### 소문자 변환 전처리

In [ ]:
corpus = []
for i in range(len(clean_sheet['text'])):
    review = clean_sheet['text'][i].lower() #lower case
    corpus.append(review)
clean_sheet['text'] = corpus

### 띄어쓰기 처리

In [ ]:
def re_spacing(text):
    result = text.strip()
    while "  " in result:
        result = result.replace("  ", " ")
    return result

In [ ]:
clean_sheet['text'] = clean_sheet['text'].apply(lambda x : re_spacing(x))

In [ ]:
clean_sheet

### 전처리 완료 후 DataFrame으로 변환

In [ ]:
final_sheet = clean_sheet.text
final_sheet = final_sheet.reset_index(drop=True)

In [ ]:
final_sheet = pd.DataFrame(final_sheet)

In [ ]:
final_sheet

### CSV 파일로 저장

In [ ]:
clean_sheet.to_csv('직장인_크롤링_전처리_데이터.csv', encoding = 'utf-8-sig') #파일명 수정(전처리) 필요